In [24]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from pathlib import Path
import textwrap
import cga_utils
import math
import rules_stats
from typing import Iterable, List, Optional, Tuple
import hdbscan
from sklearn.feature_extraction import DictVectorizer
import cluster_features
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
def pattern_freq(train, dev, test):
    res = {}
    for i, r in train.iterrows():        
        cp =  r['calc_pattern']
        
        if cp.endswith(")*#"):
            cp = cp[1:-3]
            
        if cp not in res:
            res[cp] = {'train': 0, 'dev' : 0 , 'test': 0}
            
        res[cp]['train']  = res[cp]['train'] + 1
        
    for i, r in dev.iterrows():
        
        cp =  r['calc_pattern']
        if cp.endswith(")*#"):
            cp = cp[1:-3]
            
        if cp not in res:
            res[cp] = {'train': 0, 'dev' : 0 , 'test': 0}
        res[cp]['dev']  = res[cp]['dev'] + 1
        
    
    for i, r in test.iterrows():
        cp =  r['calc_pattern']
        
        if cp.endswith(")*#"):
            cp = cp[1:-3]
            
        if cp not in res:
            res[cp] = {'train': 0, 'dev' : 0 , 'test': 0}
            
        res[cp]['test']  = res[cp]['test'] + 1


    res = pd.DataFrame.from_dict(res, orient="index")
    res["delta_dev_test"] = res['test'] - res['dev']
    res["sum"] = res['train'] + res['test'] + res['dev']
    return res
traindf = pd.read_csv('res/q3inst_v18_train.csv')
devdf = pd.read_csv('res/gemma3_v18.csv')
#devdf = pd.read_csv('../tab_small/res/e38_18r.csv')
testdf = pd.read_csv('res/q3_v18_test.csv')

pattern_freq(traindf,devdf, testdf ).query('sum > 5').sort_values(by="dev", ascending=False)[["dev","test"]]

,dev,test
#-#,134,142
(#-#)/#,100,119
(#+#)/#,89,53
#/#,49,50
(#+#+#)/#,26,19
#+#,21,15
#/#-#,9,7
-#-#,8,0
#%-#%,7,6
-(#+#)/#,5,6


In [140]:
def pattern_error_change(err_before, err_after):
    res = {}
    for i, r18 in err_before.iterrows():
        if r18['calc_pattern'] not in res:
            res[r18['calc_pattern']] = {'before' : 0 , 'after': 0}
        if not r18['exact_match']:
            res[r18['calc_pattern']]['before']  = res[r18['calc_pattern']]['before'] + 1
    
    for i, r21 in err_after.iterrows():
        if r21['calc_pattern'] not in res:
            res[r21['calc_pattern']] = {'before' : 0 , 'after': 0}
        
        if  not r21['exact_match']:
            res[r21['calc_pattern']]['after']  = res[r21['calc_pattern']]['after'] + 1
            
    res
    res = pd.DataFrame.from_dict(res, orient="index")
    res["delta"] = res['after'] - res['before']
    return res

def pattern_EM_change(err_before, err_after):
    res = {}
    for i, r18 in err_before.iterrows():
        if r18['calc_pattern'] not in res:
            res[r18['calc_pattern']] = {'before' : 0 , 'after': 0, 'cnt': 0}
        res[r18['calc_pattern']]['cnt']  = res[r18['calc_pattern']]['cnt'] + 1
        if r18['exact_match']:
            res[r18['calc_pattern']]['before']  = res[r18['calc_pattern']]['before'] + 1
    
    for i, r21 in err_after.iterrows():
        #if r21['calc_pattern'] not in res:
        #    res[r21['calc_pattern']] = {'before' : 0 , 'after': 0}
        
        if  r21['exact_match']:
            res[r21['calc_pattern']]['after']  = res[r21['calc_pattern']]['after'] + 1
            
    res
    res = pd.DataFrame.from_dict(res, orient="index")
    res["delta"] = res['after'] - res['before']
    res['before%'] = res['before'] / res['cnt'] 
    res['after%'] = res['after'] / res['cnt'] 
    res['delta%'] = res['after%'] - res['before%']
    return res

# Qwen3

In [118]:
base = pd.read_csv('../tab_small/res/e38_18r.csv')
final = pd.read_csv('../tab_small/res/e38_29.csv')
pattern_EM_change(base, final).sort_values(by="delta", ascending=False)

,before,after,cnt,delta
(#-#)/#,61,94,100,33
(#+#)/#,62,69,89,7
#/#-#,6,8,9,2
(-#-(-#))/-#,0,2,2,2
[(#+#)/#]-[(-#)+(-#)]/#,0,1,1,1
#+(#)+#,0,1,1,1
(-#-#)/#,1,2,2,1
(-#)-(-#),0,1,1,1
-#/#-#,0,1,1,1
(#+#+#+#+#+#+#)/#,0,0,1,0


In [94]:
rules_stats.rule_gate_decision(base, final, use_bootstrap=False)

RuleDecision(accepted=True, reason='OK', summary={'N': 497, 'b_improved': 69, 'c_degraded': 31, 'delta_pp': 7.64587525150906, 'p_mcnemar_exact': 0.00018314322488235366, 'p_mcnemar_chi2cc': 0.0002155994669548189, 'criteria': {'mcnemar_and_delta': True}})

In [143]:
base = pd.read_csv('res/q3_v18_test.csv')
final = pd.read_csv('res/q3_v29_test.csv')
rules_stats.rule_gate_decision(base, final, use_bootstrap=False)

RuleDecision(accepted=True, reason='OK', summary={'N': 471, 'b_improved': 71, 'c_degraded': 25, 'delta_pp': 9.766454352441611, 'p_mcnemar_exact': 2.8675552580223223e-06, 'p_mcnemar_chi2cc': 4.373522667910912e-06, 'criteria': {'mcnemar_and_delta': True}})

In [146]:
pattern_EM_change(base, final).query('cnt > 4').sort_values(by="delta%", ascending=False)

,before,after,cnt,delta,before%,after%,delta%
((#-#)/#)*#,53,92,113,39,0.469027,0.814159,0.345133
-#-(-#),4,5,7,1,0.571429,0.714286,0.142857
(#/#)*#,20,26,42,6,0.476190,0.619048,0.142857
#+#,9,11,15,2,0.600000,0.733333,0.133333
(#/#-#)*#,5,5,7,0,0.714286,0.714286,0.000000
(#-#)/#,5,5,6,0,0.833333,0.833333,0.000000
-(#+#)/#,5,5,6,0,0.833333,0.833333,0.000000
#-#,95,93,142,-2,0.669014,0.654930,-0.014085
(#+#)/#,31,30,52,-1,0.596154,0.576923,-0.019231
(#+#+#)/#,12,11,19,-1,0.631579,0.578947,-0.052632


# Gemma3N 

In [78]:
ar18 = pd.read_csv('res/gemma3n_v18.csv')
ar21 = pd.read_csv('res/gemma3n_v21_r1.csv')

In [82]:
pattern_error_change(ar18, ar21).sort_values(by="delta")

,before,after,delta
(#-#)/#,51,28,-23
#-#,74,67,-7
#/#,32,29,-3
#+#,17,15,-2
#/#-#,5,3,-2
(#+#+#)/(#-#+#),1,0,-1
(-#-(-#))/-#,2,1,-1
-#/#-#,1,0,-1
#%-#%,3,2,-1
#+#+#,4,3,-1


In [80]:
rules_stats.rule_gate_decision(ar18, ar21, use_bootstrap=False)

RuleDecision(accepted=True, reason='OK', summary={'N': 497, 'b_improved': 66, 'c_degraded': 24, 'delta_pp': 8.450704225352107, 'p_mcnemar_exact': 1.0902049191089246e-05, 'p_mcnemar_chi2cc': 1.5477584790146715e-05, 'criteria': {'mcnemar_and_delta': True}})

In [95]:
ar18 = pd.read_csv('res/g3n_v18_test.csv')
ar21 = pd.read_csv('res/g3n_v21_test.csv')
rules_stats.rule_gate_decision(ar18, ar21, use_bootstrap=False)

RuleDecision(accepted=True, reason='OK', summary={'N': 471, 'b_improved': 72, 'c_degraded': 26, 'delta_pp': 9.766454352441611, 'p_mcnemar_exact': 3.6886460339466607e-06, 'p_mcnemar_chi2cc': 5.475650002972898e-06, 'criteria': {'mcnemar_and_delta': True}})

# Gemma3 V1

In [90]:
base = pd.read_csv('res/gemma3_v18.csv')
final = pd.read_csv('res/gemma3_v19a.csv')


In [91]:
pattern_error_change(base, final).sort_values(by="delta")

,before,after,delta
(#+#)/#,64,47,-17
#-#,76,74,-2
-#-(#),1,0,-1
(#+#+#)/#,19,18,-1
[(-#)-(-#)]/-#,1,0,-1
#/#,30,29,-1
#-(-#),2,2,0
(#/#)-(#/#),0,0,0
(-#)-(-#),0,0,0
(#+#+#)/(#-#+#),0,0,0


In [92]:
rules_stats.rule_gate_decision(base, final, use_bootstrap=False)

RuleDecision(accepted=False, reason='McNemar p>=alpha vagy ΔEM túl kicsi', summary={'N': 497, 'b_improved': 55, 'c_degraded': 36, 'delta_pp': 3.8229376257545242, 'p_mcnemar_exact': 0.058573528410188855, 'p_mcnemar_chi2cc': 0.059172067825859376, 'criteria': {'mcnemar_and_delta': False}})

In [100]:
base = pd.read_csv('res/g3_v18_test_v1.csv')
final = pd.read_csv('res/g3_v19a_test_v1.csv')
rules_stats.rule_gate_decision(base, final, use_bootstrap=False)

RuleDecision(accepted=False, reason='McNemar p>=alpha vagy ΔEM túl kicsi', summary={'N': 471, 'b_improved': 52, 'c_degraded': 38, 'delta_pp': 2.972399150743099, 'p_mcnemar_exact': 0.17024239504607366, 'p_mcnemar_chi2cc': 0.17058693287144122, 'criteria': {'mcnemar_and_delta': False}})

# Gemma3 V2

In [85]:
base = pd.read_csv('res/gemma3_v18.csv')
final = pd.read_csv('res/gemma3_v19d_r1.csv')

In [86]:
pattern_error_change(base, final)

,before,after,delta
#-#,76,67,-9
(#-#)/#,28,31,3
(#+#)/#,64,49,-15
[(#+#)/#]-[(#+#)/#],4,4,0
#/#,30,33,3
((#-#)+(#-#))/#,1,1,0
-#-#,6,5,-1
(#%+#%+#%)/#,2,2,0
(#+#+#)/#,19,22,3
#/#-#,3,4,1


In [87]:
rules_stats.rule_gate_decision(base, final, use_bootstrap=False)

RuleDecision(accepted=False, reason='McNemar p>=alpha vagy ΔEM túl kicsi', summary={'N': 497, 'b_improved': 50, 'c_degraded': 36, 'delta_pp': 2.8169014084507005, 'p_mcnemar_exact': 0.16057841913397977, 'p_mcnemar_chi2cc': 0.1609671969711517, 'criteria': {'mcnemar_and_delta': False}})

In [101]:
base = pd.read_csv('res/g3_v18_test_v2.csv')
final = pd.read_csv('res/g3_v19d_test_v2.csv')
rules_stats.rule_gate_decision(base, final, use_bootstrap=False)

RuleDecision(accepted=False, reason='McNemar p>=alpha vagy ΔEM túl kicsi', summary={'N': 471, 'b_improved': 43, 'c_degraded': 41, 'delta_pp': 0.424628450106157, 'p_mcnemar_exact': 0.9132023622245978, 'p_mcnemar_chi2cc': 0.9131160800723743, 'criteria': {'mcnemar_and_delta': False}})